In [1]:
import satpy
from satpy import Scene
from glob import glob
from satpy.writers import get_enhanced_image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pyresample import create_area_def
from pathlib import Path
import xarray as xr

### Process MODIS 1b data using satpy

In [30]:
# specify folder containing set of MODIS hdf files

#folder = "AquaHkmLabSea2022b"
#folder = 'AquaQkmLabSeaOctNov22'
folder = "Aqua1km"
file_loc = "/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/" + folder
filenames = glob(file_loc + "/*.hdf")

In [31]:
filenames[0]

'/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1km/MYD021KM.A2022297.1655.061.2022298192527.hdf'

In [36]:
# check available channels and composites

scn = Scene(reader='modis_l1b', filenames=[filenames[0]])
print(scn.available_dataset_names())
print(scn.available_composite_names())


Too many possible datasets to load for DataQuery(wavelength=0.67)


['1', '10', '11', '12', '13hi', '13lo', '14hi', '14lo', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6', '7', '8', '9', 'latitude', 'longitude', 'satellite_azimuth_angle', 'satellite_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle']
['airmass', 'ash', 'day_essl_colorized_low_level_moisture', 'day_essl_low_level_moisture', 'day_microphysics', 'dust', 'essl_colorized_low_level_moisture', 'essl_low_level_moisture', 'fog', 'green_snow', 'ir108_3d', 'ir_cloud_day', 'natural_color', 'natural_color_raw', 'natural_with_night_fog', 'night_fog', 'ocean_color', 'overview', 'snow', 'true_color', 'true_color_crefl', 'true_color_thin', 'true_color_uncorrected']


In [4]:
scn.load(['1', '4', '3', 'true_color_uncorrected'])
scn.save_datasets(filename="test.nc",writer="cf",
                    datasets=['1', '4', '3', 'true_color_uncorrected'])

Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'


[None]

In [44]:
ds = xr.open_dataset("test.nc")
ds

<xarray.Dataset> Size: 352MB
Dimensions:                 (y: 4060, x: 2708, bands: 3)
Coordinates:
    longitude               (y, x) float32 44MB ...
    latitude                (y, x) float32 44MB ...
  * bands                   (bands) <U1 12B 'R' 'G' 'B'
Dimensions without coordinates: y, x
Data variables:
    CHANNEL_1               (y, x) float32 44MB ...
    CHANNEL_3               (y, x) float32 44MB ...
    CHANNEL_4               (y, x) float32 44MB ...
    true_color_uncorrected  (bands, y, x) float32 132MB ...
Attributes:
    history:      Created by pytroll/satpy on 2024-06-26 12:35:16.158153
    Conventions:  CF-1.7

In [22]:
scn.load(['1'])
scn['1']

<xarray.DataArray 'getitem-ee462418a8607238bc307661a755c8b4' (y: 8120, x: 5416)> Size: 176MB
dask.array<mul, shape=(8120, 5416), dtype=float32, chunksize=(6160, 5416), chunktype=numpy.ndarray>
Coordinates:
    crs      object 8B +proj=longlat +ellps=WGS84 +type=crs
Dimensions without coordinates: y, x
Attributes: (12/18)
    name:                 1
    resolution:           250
    calibration:          reflectance
    coordinates:          ('longitude', 'latitude')
    wavelength:           0.645 µm (0.62-0.67 µm)
    file_type:            hdf_eos_data_250m
    ...                   ...
    start_time:           2022-10-30 16:15:00
    end_time:             2022-10-30 16:20:00
    reader:               modis_l1b
    area:                 Shape: (8120, 5416)\nLons: <xarray.DataArray 'getit...
    _satpy_id:            DataID(name='1', wavelength=WavelengthRange(min=0.6...
    ancillary_variables:  []

In [33]:
scn.load(['latitude'])
scn['latitude']

<xarray.DataArray 'getitem-c9a9b5c8ecb3a11bf7b526c5260e052b' (y: 2030, x: 1354)> Size: 11MB
dask.array<add, shape=(2030, 1354), dtype=float32, chunksize=(1540, 1354), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x
Attributes: (12/13)
    standard_name:        latitude
    units:                degree
    platform_name:        EOS-Aqua
    sensor:               modis
    rows_per_scan:        10
    start_time:           2022-10-24 16:55:00
    ...                   ...
    reader:               modis_l1b
    name:                 latitude
    resolution:           1000
    modifiers:            ()
    _satpy_id:            DataID(name='latitude', resolution=1000, modifiers=())
    ancillary_variables:  []

In [34]:
scn['latitude'].values


array([[51.497078, 51.497128, 51.49719 , ..., 46.97993 , 46.96229 ,
        46.944542],
       [51.515182, 51.515182, 51.515182, ..., 46.996277, 46.978672,
        46.960945],
       [51.533302, 51.533245, 51.533184, ..., 47.012623, 46.99506 ,
        46.977367],
       ...,
       [69.382286, 69.38563 , 69.38885 , ..., 62.168   , 62.1396  ,
        62.11093 ],
       [69.400314, 69.40361 , 69.40678 , ..., 62.181442, 62.153072,
        62.12443 ],
       [69.41834 , 69.421585, 69.424706, ..., 62.19487 , 62.166523,
        62.137913]], dtype=float32)

#### Save slices as png images

In [4]:
def save_slice(file, filepath):
    '''saves a 1024 wide slice of the image to remove distorted areas (as Geiss 2024)'''
    # open file
    scn = Scene(reader='modis_l1b', filenames=[file])
    scn.load(['true_color_uncorrected'])
    # slice off the edges
    scn_slice = scn[:,:,843:1867] # composites have three dimensions where x is the third
    # flip the image
    scn_slice =  scn_slice[:,::-1,::-1]
    # save the sliced image
    scn_slice.save_datasets(filename=filepath+"/{name}{start_time}.png",writer="simple_image")
    

In [5]:
# make filepath for image slices within data folder

sliced_loc = file_loc + "/sliced"
filepath = Path(sliced_loc)
filepath.mkdir(exist_ok=True, parents=True)

In [6]:
for file in filenames:
    save_slice(file, str(filepath));

Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file ty

#### Save rgb and true colour composite data as netcdf files

NB this results in very large data files (500MB each), is it totally necessary? Could I create image tiles and write to csv directly from granules using satpy?

In [40]:
# save true colour composite as netcdf (whole image as can't slice lon and lat)
    
def save_nc(file, filepath):
    # open file
    scn = Scene(reader='modis_l1b', filenames=[file])
    scn.load(['1', '4', '3', 'true_color_uncorrected'])
    scn.save_datasets(filename=filepath+"/{name}{start_time}.nc",writer="cf",
                      datasets=['1', '3', '4', 'true_color_uncorrected'])

In [41]:
# make filepath for netcdf files within data folder

nc_loc = file_loc + "/true_colour_netcdf"
nc_filepath = Path(nc_loc)
nc_filepath.mkdir(exist_ok=True, parents=True)

In [42]:
for file in filenames:
    save_nc(file, str(nc_filepath));

Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file type '['hdf_eos_geo']' not found or loaded for 'solar_zenith_angle'
Required file ty

#### Save rgb bands to netcdf files
Red = Band 1, Green = Band 4, Blue = Band 3

In [ ]:
def save_nc(file, filepath):
    # open file
    scn = Scene(reader='modis_l1b', filenames=[file])
    scn.load(['1', '4', '3'])
    scn.save_datasets(filename=filepath+"/{name}{start_time}.nc",writer="cf",
                      datasets=['1', '4', '3'])

### Can I make tiles directly from hdf files?

In [ ]:
TRIPLET_TILE_IDENTIFIER_FORMAT = "{triplet_id:05d}_{tile_type}"
TRIPLET_TILE_FILENAME_FORMAT = f"{TRIPLET_TILE_IDENTIFIER_FORMAT}.png"

In [ ]:
tile_nx = tile_ny = 256
img_tiles = []

for file in filenames:

    # open file
    scn = Scene(reader='modis_l1b', filenames=[file])
    scn.load(['true_color_uncorrected'])
    # slice off the edges
    scn_slice = scn[:,:,843:1867] # composites have three dimensions where x is the third
    # flip the image
    scn_slice =  scn_slice[:,::-1,::-1]

    # now we have a slice, want to crop to lots of tiles


    img = imageio.imread(file)
    img = img[:,:,:3] # last term gets rid of alpha channel
    
    # size of a slice
    ny, nx, _ = 4060, 1024

    for i in range(0, nx-tile_nx, 64):
        for j in range(0, ny-tile_ny, 64):
            img_tile = img[j:j+tile_ny, i:i+tile_nx, :]
            img_tiles.append(img_tile)

len(img_tiles)

In [ ]:
# Then need to save some of the tiles as images, along with lat lon info in a csv